# Bloque 1: Importaciones y Configuración

- Propósito: Cargar todas las "cajas de herramientas" (librerías) que usaremos

1) librosa: Para cargar y analizar los audios.
2) sklearn: Para crear y entrenar el modelo de Machine Learning.
3) joblib: Para guardar nuestro modelo entrenado en un archivo.

También definimos las rutas:
1) DATASET_BASE_PATH: La carpeta donde están los audios de entrenamiento.
2) CLASES: Los nombres de las subcarpetas (nuestras categorías).

In [1]:
# Bloque 1: Importaciones y configuración
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

DATASET_BASE_PATH = "audios_proyecto/"
CLASES = ["pequeno", "mediano", "grande"]

# Bloque 2: Función de Extracción de Características

- Propósito: Definir la "receta" para convertir un archivo de audio en números.

Esta función es el cerebro del proceso:
1) Carga el audio con `librosa.load()`.
2) Calcula 40 "MFCCs" (Coeﬁcientes Cepstrales en las Frecuencias de Mel).
3) Calcula la media de esos MFCCs para resumir el audio en un solo vector (lista) de 40 números, sin importar cuánto dure el audio.

In [2]:
# Bloque 2: Función de extracción de características
def extract_features(file_path, n_mfcc=40):
    try:
        y, sr = librosa.load(file_path, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        features_vector = np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"Error cargando {file_path}: {e}")
        return None
    return features_vector

# Bloque 3: Carga y Extracción de Datos

- Propósito: Leer todas las carpetas de audio y preparar los datos para entrenar.

1) Crea listas vacías ("features" y "labels") para guardar los datos.
2) Recorre cada carpeta de "CLASES" (pequeno, mediano, grande).
3) Para cada archivo de audio (.wav) dentro de esas carpetas:
    - Llama a la función "extract_features" (del Bloque 2)
    - Guarda el vector de números en la lista "features" (la "x")
    - Guarda la etiqueta (0, 1 o 2) en la lista "labels" (la "y")
4) Al final, convierte esas listas de Python en arrays de NumPy (que es lo que "sklearn" necesita para entrenar)

In [3]:
# Bloque 3: Carga y extracción de datos
features = [] 
labels = []   

print("1. Cargando y extrayendo features...")
for i, clase in enumerate(CLASES):
    folder_path = os.path.join(DATASET_BASE_PATH, clase)
    print(f"   -> Procesando clase: {clase} (etiqueta: {i})")
    
    if not os.path.isdir(folder_path):
        print(f"   ERROR: Carpeta no encontrada: {folder_path}.")
        continue
        
    for filename in os.listdir(folder_path):
        if filename.endswith(('.wav', '.mp3')):
            file_path = os.path.join(folder_path, filename)
            data = extract_features(file_path) # Usa la función del Bloque 2
            
            if data is not None:
                features.append(data)
                labels.append(i)

if not features:
    print("\n¡ADVERTENCIA! No se cargaron muestras. Verifica tus carpetas.")
else:
    print(f"\nTotal de muestras cargadas: {len(features)}")
    X = np.array(features)
    y = np.array(labels)

1. Cargando y extrayendo features...
   -> Procesando clase: pequeno (etiqueta: 0)
   -> Procesando clase: mediano (etiqueta: 1)
   -> Procesando clase: grande (etiqueta: 2)

Total de muestras cargadas: 28


# Bloque 4: Entrenamiento y guardado del modelo

- Propósito: Entrenar el modelo de IA y guardarlo

1) Divide los datos: 80% para "estudiar" (entrenar) y 20% para "probar" (testear).
2) Crea el modelo (`RandomForestClassifier`, un modelo simple y efectivo).
3) Entrena el modelo usando los datos de "estudio" (`model.fit`).
4) Evalúa qué tan bien aprendió usando los datos de "prueba" y muestra la precisión.
5) Guarda el modelo ya entrenado en el archivo 'clasificador_vehiculos.joblib'. Este archivo es el "cerebro" que usará el Notebook 2.

In [4]:
# Bloque 4: Entrenamiento y Guardado del Modelo
if 'X' in locals() and len(X) > 0:
    # Dividir datos
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenar
    print("2. Entrenando modelo RandomForest...")
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Evaluar
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"   Precisión en datos de prueba: {accuracy * 100:.2f}%")

    # Guardar
    joblib.dump(model, 'clasificador_vehiculos.joblib')
    print("\n3. Modelo guardado exitosamente como 'clasificador_vehiculos.joblib'")
else:
    print("\nNo se pudo entrenar el modelo (no se cargaron datos).")

2. Entrenando modelo RandomForest...
   Precisión en datos de prueba: 66.67%

3. Modelo guardado exitosamente como 'clasificador_vehiculos.joblib'


* Al final de la ejecucion se creara el archivo "clasificador_vehiculos.joblib" el cual se utilizara en "02_Conteo_Aplicacion.ipynb" para clasificar el tipo de vehiculo que se escucha en el audio a evaluar.